# Day9_3: 통계 종합 실습 (Statistics Comprehensive Practice)

## 📚 학습 목표

**Part 1: 통계 분석 통합**
1. Day 9 전체 학습 내용(확률분포, 가설검정, 회귀분석) 통합 복습하기
2. 실전 데이터셋으로 EDA(탐색적 데이터 분석) 수행하기
3. 정규성 검정을 통해 적절한 분석 방법 선택하기
4. 상관분석으로 변수 간 관계 파악하기
5. t-test와 ANOVA로 그룹 간 차이 검정하기

**Part 2: 종합 분석 프로젝트**
1. 비즈니스 가설을 통계적 가설로 변환하기
2. 회귀모델로 예측 및 변수 중요도 파악하기
3. Plotly로 분석 결과 인터랙티브 시각화하기
4. 분석 결과를 비즈니스 인사이트로 해석하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 통계 분석 통합 | 복합적인 비즈니스 문제 해결 | 마케팅 효과 + 고객 세분화 동시 분석 |
| 가설 기반 분석 | 데이터 기반 의사결정 | "신규 기능이 매출을 올렸나?" 검증 |
| 회귀 모델링 | 핵심 변수 파악 및 예측 | 매출에 가장 큰 영향을 주는 요인 |
| 결과 시각화 | 이해관계자 커뮤니케이션 | 경영진에게 분석 결과 보고 |

**분석가 관점**: 실제 업무에서는 단일 통계 기법이 아닌 여러 기법을 조합하여 복합적인 질문에 답해야 합니다!

---

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

---

# Part 1: 통계 분석 통합 복습

---

## 1.1 종합 분석 프레임워크

### 통계 분석 5단계 워크플로우

```
1. 비즈니스 질문 정의
       ↓
2. 데이터 탐색 (EDA)
       ↓
3. 분포 확인 & 정규성 검정
       ↓
4. 적절한 통계 검정 수행
       ↓
5. 결과 해석 & 시각화
```

### 분석 기법 선택 가이드

| 질문 유형 | 분석 기법 | Python 함수 |
|----------|----------|-------------|
| 평균이 기준값과 다른가? | 단일 표본 t-test | `stats.ttest_1samp()` |
| 두 그룹 평균이 다른가? | 독립 표본 t-test | `stats.ttest_ind()` |
| 전후 차이가 있는가? | 대응 표본 t-test | `stats.ttest_rel()` |
| 3개 이상 그룹 차이? | ANOVA | `stats.f_oneway()` |
| 범주형 변수 관련성? | Chi-square | `stats.chi2_contingency()` |
| 두 변수 관계? | 상관분석 | `stats.pearsonr()` |
| 예측 모델 필요? | 회귀분석 | `smf.ols()` |

---

## 1.2 실전 데이터셋: 온라인 쇼핑몰 고객 데이터

### 데이터셋 소개

온라인 쇼핑몰의 고객 행동 및 구매 데이터를 분석합니다.

**비즈니스 질문들**:
1. 남성과 여성의 구매 금액에 차이가 있는가?
2. 연령대별 구매 패턴이 다른가?
3. 방문 횟수와 구매 금액은 관련이 있는가?
4. 어떤 요인이 구매 금액에 가장 큰 영향을 미치는가?

In [ ]:
# 실전 데이터셋 생성
np.random.seed(42)

n_customers = 200

# 고객 데이터 생성
customers = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'gender': np.random.choice(['남성', '여성'], n_customers, p=[0.45, 0.55]),
    'age': np.random.randint(20, 60, n_customers),
    'membership': np.random.choice(['일반', '실버', '골드', 'VIP'], n_customers, p=[0.4, 0.3, 0.2, 0.1]),
    'visit_count': np.random.poisson(8, n_customers),  # 월 평균 방문 횟수
    'avg_session_time': np.random.normal(15, 5, n_customers).clip(3, 40),  # 평균 체류시간(분)
    'items_viewed': np.random.poisson(12, n_customers),  # 조회한 상품 수
})

# 연령대 추가
customers['age_group'] = pd.cut(customers['age'], bins=[19, 29, 39, 49, 60], labels=['20대', '30대', '40대', '50대'])

# 구매 금액 생성 (여러 요인에 영향을 받음)
base_purchase = 50000
gender_effect = (customers['gender'] == '여성').astype(int) * 15000
membership_effect = customers['membership'].map({'일반': 0, '실버': 20000, '골드': 50000, 'VIP': 100000})
visit_effect = customers['visit_count'] * 5000
session_effect = customers['avg_session_time'] * 1000
noise = np.random.normal(0, 20000, n_customers)

customers['purchase_amount'] = (base_purchase + gender_effect + membership_effect + 
                                visit_effect + session_effect + noise).clip(10000, 500000).astype(int)

# 구매 여부 (구매 금액 기반)
customers['purchased'] = (customers['purchase_amount'] > 80000).astype(int)

print(f"데이터셋 크기: {customers.shape}")
print("\n처음 5행:")
customers.head()

In [ ]:
# 기초 통계량 확인
print("=" * 60)
print("기초 통계량")
print("=" * 60)
print(customers.describe())

print("\n범주형 변수 분포:")
print(f"\n성별:\n{customers['gender'].value_counts()}")
print(f"\n멤버십:\n{customers['membership'].value_counts()}")
print(f"\n연령대:\n{customers['age_group'].value_counts().sort_index()}")

### 💡 실무 예시: EDA 체크리스트

1. **데이터 크기 확인**: 행, 열 수
2. **결측치 확인**: `isnull().sum()`
3. **데이터 타입 확인**: `dtypes`
4. **기초 통계량**: `describe()`
5. **분포 시각화**: 히스토그램, 박스플롯

In [ ]:
# 구매 금액 분포 시각화
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('구매 금액 분포', '성별 구매 금액 비교'))

# 히스토그램
fig.add_trace(
    go.Histogram(x=customers['purchase_amount'], nbinsx=30, name='전체'),
    row=1, col=1
)

# 성별 박스플롯
for gender in ['남성', '여성']:
    data = customers[customers['gender'] == gender]['purchase_amount']
    fig.add_trace(
        go.Box(y=data, name=gender),
        row=1, col=2
    )

fig.update_layout(height=400, width=900, title_text="구매 금액 EDA", showlegend=False)
fig.update_xaxes(title_text="구매 금액 (원)", row=1, col=1)
fig.update_yaxes(title_text="빈도", row=1, col=1)
fig.update_yaxes(title_text="구매 금액 (원)", row=1, col=2)
fig.show()

---

## 1.3 정규성 검정

### 왜 정규성 검정이 필요한가?

많은 통계 검정(t-test, ANOVA 등)은 **데이터가 정규분포를 따른다고 가정**합니다.

- **정규분포 O** → 모수적 검정 (t-test, ANOVA)
- **정규분포 X** → 비모수적 검정 (Mann-Whitney, Kruskal-Wallis)

### 정규성 검정 방법

1. **Shapiro-Wilk 검정**: 가장 많이 사용 (n < 5000 권장)
2. **Kolmogorov-Smirnov 검정**: 큰 표본에 적합
3. **Q-Q plot**: 시각적 확인

In [ ]:
# Shapiro-Wilk 정규성 검정
def test_normality(data, name):
    """Shapiro-Wilk 검정으로 정규성 확인"""
    stat, p_value = stats.shapiro(data)
    print(f"\n{name}:")
    print(f"  Shapiro-Wilk 통계량: {stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    if p_value < 0.05:
        print(f"  → ❌ 정규분포가 아님 (p < 0.05)")
        return False
    else:
        print(f"  → ✅ 정규분포 가정 충족 (p >= 0.05)")
        return True

print("=" * 60)
print("정규성 검정 결과")
print("=" * 60)

# 주요 변수 정규성 검정
test_normality(customers['purchase_amount'], '구매 금액')
test_normality(customers['avg_session_time'], '평균 체류시간')
test_normality(customers['visit_count'], '방문 횟수')

In [ ]:
# Q-Q Plot으로 정규성 시각적 확인
from scipy.stats import probplot

fig = make_subplots(rows=1, cols=2, subplot_titles=('구매 금액 Q-Q Plot', '체류시간 Q-Q Plot'))

# 구매 금액 Q-Q plot
qq_data1 = probplot(customers['purchase_amount'])
fig.add_trace(
    go.Scatter(x=qq_data1[0][0], y=qq_data1[0][1], mode='markers', name='데이터'),
    row=1, col=1
)
# 이론적 직선
x_line = np.linspace(min(qq_data1[0][0]), max(qq_data1[0][0]), 100)
y_line = qq_data1[1][0] * x_line + qq_data1[1][1]
fig.add_trace(
    go.Scatter(x=x_line, y=y_line, mode='lines', name='이론선', line=dict(color='red')),
    row=1, col=1
)

# 체류시간 Q-Q plot
qq_data2 = probplot(customers['avg_session_time'])
fig.add_trace(
    go.Scatter(x=qq_data2[0][0], y=qq_data2[0][1], mode='markers', name='데이터', showlegend=False),
    row=1, col=2
)
x_line2 = np.linspace(min(qq_data2[0][0]), max(qq_data2[0][0]), 100)
y_line2 = qq_data2[1][0] * x_line2 + qq_data2[1][1]
fig.add_trace(
    go.Scatter(x=x_line2, y=y_line2, mode='lines', name='이론선', line=dict(color='red'), showlegend=False),
    row=1, col=2
)

fig.update_layout(height=400, width=900, title_text="Q-Q Plot (정규성 시각적 확인)")
fig.update_xaxes(title_text="이론적 분위수", row=1, col=1)
fig.update_yaxes(title_text="표본 분위수", row=1, col=1)
fig.update_xaxes(title_text="이론적 분위수", row=1, col=2)
fig.update_yaxes(title_text="표본 분위수", row=1, col=2)
fig.show()

print("\n💡 해석: 데이터 점들이 빨간 직선에 가까울수록 정규분포에 가깝습니다.")

---

## 1.4 그룹 비교 분석

### 질문 1: 남성과 여성의 구매 금액에 차이가 있는가?

**가설 설정**:
- H₀: 남성 평균 구매 금액 = 여성 평균 구매 금액
- H₁: 남성 평균 구매 금액 ≠ 여성 평균 구매 금액

In [ ]:
# 성별 구매 금액 비교
male_purchase = customers[customers['gender'] == '남성']['purchase_amount']
female_purchase = customers[customers['gender'] == '여성']['purchase_amount']

print("=" * 60)
print("성별 구매 금액 비교 분석")
print("=" * 60)

# 기초 통계
print(f"\n남성 (n={len(male_purchase)}):")
print(f"  평균: {male_purchase.mean():,.0f}원")
print(f"  표준편차: {male_purchase.std():,.0f}원")

print(f"\n여성 (n={len(female_purchase)}):")
print(f"  평균: {female_purchase.mean():,.0f}원")
print(f"  표준편차: {female_purchase.std():,.0f}원")

print(f"\n차이: {female_purchase.mean() - male_purchase.mean():,.0f}원")

# 독립 표본 t-test
t_stat, p_value = stats.ttest_ind(male_purchase, female_purchase)

print(f"\n통계 검정 (독립 표본 t-test):")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.4f}")

print("\n" + "=" * 60)
if p_value < 0.05:
    print("✅ 결론: 남성과 여성의 구매 금액에 통계적으로 유의한 차이가 있습니다!")
else:
    print("❌ 결론: 남성과 여성의 구매 금액에 유의한 차이가 없습니다.")
print("=" * 60)

### 질문 2: 연령대별 구매 금액에 차이가 있는가?

**가설 설정**:
- H₀: 모든 연령대의 평균 구매 금액이 같다
- H₁: 적어도 하나의 연령대 구매 금액이 다르다

**분석 방법**: ANOVA (3개 이상 그룹 비교)

In [ ]:
# 연령대별 구매 금액 비교 (ANOVA)
age_groups = customers.groupby('age_group')['purchase_amount']

print("=" * 60)
print("연령대별 구매 금액 비교 분석")
print("=" * 60)

# 연령대별 기초 통계
for name, group in age_groups:
    print(f"\n{name} (n={len(group)}):")
    print(f"  평균: {group.mean():,.0f}원")
    print(f"  표준편차: {group.std():,.0f}원")

# ANOVA 검정
groups = [group.values for name, group in age_groups]
f_stat, p_value = stats.f_oneway(*groups)

print(f"\n통계 검정 (One-Way ANOVA):")
print(f"  F-statistic: {f_stat:.4f}")
print(f"  p-value: {p_value:.4f}")

print("\n" + "=" * 60)
if p_value < 0.05:
    print("✅ 결론: 연령대별 구매 금액에 통계적으로 유의한 차이가 있습니다!")
    # 최고/최저 연령대 확인
    means = age_groups.mean().sort_values(ascending=False)
    print(f"   최고: {means.index[0]} ({means.iloc[0]:,.0f}원)")
    print(f"   최저: {means.index[-1]} ({means.iloc[-1]:,.0f}원)")
else:
    print("❌ 결론: 연령대별 구매 금액에 유의한 차이가 없습니다.")
print("=" * 60)

In [ ]:
# 연령대별 구매 금액 시각화
fig = px.box(
    customers, 
    x='age_group', 
    y='purchase_amount',
    color='age_group',
    title='연령대별 구매 금액 분포',
    labels={'age_group': '연령대', 'purchase_amount': '구매 금액 (원)'},
    category_orders={'age_group': ['20대', '30대', '40대', '50대']}
)

fig.update_layout(width=700, height=500, showlegend=False)
fig.show()

---

## 1.5 상관분석

### 질문 3: 방문 횟수와 구매 금액은 관련이 있는가?

**가설 설정**:
- H₀: 방문 횟수와 구매 금액 사이에 상관관계가 없다 (r = 0)
- H₁: 방문 횟수와 구매 금액 사이에 상관관계가 있다 (r ≠ 0)

In [ ]:
# 상관분석
print("=" * 60)
print("상관분석 결과")
print("=" * 60)

# 주요 변수 간 상관계수
numeric_cols = ['visit_count', 'avg_session_time', 'items_viewed', 'purchase_amount']
corr_matrix = customers[numeric_cols].corr()

print("\n상관계수 행렬:")
print(corr_matrix.round(3))

# 방문 횟수와 구매 금액 상관관계
r, p_value = stats.pearsonr(customers['visit_count'], customers['purchase_amount'])

print(f"\n방문 횟수 vs 구매 금액:")
print(f"  Pearson 상관계수: {r:.4f}")
print(f"  p-value: {p_value:.6f}")

# 상관 강도 해석
if abs(r) > 0.7:
    strength = "매우 강한"
elif abs(r) > 0.5:
    strength = "강한"
elif abs(r) > 0.3:
    strength = "중간"
elif abs(r) > 0.1:
    strength = "약한"
else:
    strength = "거의 없는"

direction = "양의" if r > 0 else "음의"

print(f"\n해석: {strength} {direction} 상관관계")
if p_value < 0.05:
    print("✅ 통계적으로 유의함")
else:
    print("❌ 통계적으로 유의하지 않음")

In [ ]:
# 상관관계 히트맵 시각화
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    aspect='auto',
    color_continuous_scale='RdBu_r',
    title='변수 간 상관계수 히트맵',
    labels={'x': '', 'y': '', 'color': '상관계수'}
)

fig.update_layout(width=600, height=500)
fig.show()

In [ ]:
# 산점도 시각화 (방문 횟수 vs 구매 금액)
fig = px.scatter(
    customers,
    x='visit_count',
    y='purchase_amount',
    color='gender',
    trendline='ols',
    title=f'방문 횟수 vs 구매 금액 (r = {r:.3f})',
    labels={'visit_count': '월 방문 횟수', 'purchase_amount': '구매 금액 (원)', 'gender': '성별'},
    width=800,
    height=500
)

fig.show()

---

# Part 2: 종합 분석 프로젝트

---

## 2.1 회귀분석으로 예측 모델 구축

### 질문 4: 어떤 요인이 구매 금액에 가장 큰 영향을 미치는가?

**분석 목표**:
- 다중 회귀분석으로 구매 금액 예측 모델 구축
- 각 변수의 영향력(회귀계수) 해석
- 모델의 설명력(R-squared) 평가

In [ ]:
# 회귀분석을 위한 데이터 준비
# 범주형 변수를 더미 변수로 변환
customers_reg = customers.copy()
customers_reg['gender_female'] = (customers_reg['gender'] == '여성').astype(int)
customers_reg['membership_silver'] = (customers_reg['membership'] == '실버').astype(int)
customers_reg['membership_gold'] = (customers_reg['membership'] == '골드').astype(int)
customers_reg['membership_vip'] = (customers_reg['membership'] == 'VIP').astype(int)

# 다중 회귀분석
formula = 'purchase_amount ~ visit_count + avg_session_time + items_viewed + gender_female + membership_silver + membership_gold + membership_vip'
model = smf.ols(formula, data=customers_reg).fit()

print("=" * 60)
print("다중 회귀분석 결과")
print("=" * 60)
print(model.summary())

In [ ]:
# 회귀계수 시각화
coef_df = pd.DataFrame({
    'variable': model.params.index[1:],  # Intercept 제외
    'coefficient': model.params.values[1:],
    'p_value': model.pvalues.values[1:]
})
coef_df['significant'] = coef_df['p_value'] < 0.05
coef_df = coef_df.sort_values('coefficient', ascending=True)

# 변수명 한글화
var_names = {
    'visit_count': '방문 횟수',
    'avg_session_time': '평균 체류시간',
    'items_viewed': '상품 조회 수',
    'gender_female': '성별(여성)',
    'membership_silver': '멤버십(실버)',
    'membership_gold': '멤버십(골드)',
    'membership_vip': '멤버십(VIP)'
}
coef_df['variable_kr'] = coef_df['variable'].map(var_names)

fig = px.bar(
    coef_df,
    y='variable_kr',
    x='coefficient',
    orientation='h',
    color='significant',
    color_discrete_map={True: 'royalblue', False: 'lightgray'},
    title='회귀계수 크기 비교 (구매 금액에 대한 영향력)',
    labels={'variable_kr': '변수', 'coefficient': '회귀계수', 'significant': '유의성'}
)

fig.update_layout(width=800, height=450, showlegend=True)
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.show()

print("\n💡 해석:")
print("- 양수: 해당 변수가 증가하면 구매 금액 증가")
print("- 음수: 해당 변수가 증가하면 구매 금액 감소")
print("- 파란색: 통계적으로 유의한 변수 (p < 0.05)")

In [ ]:
# 모델 성능 요약
print("=" * 60)
print("모델 성능 요약")
print("=" * 60)
print(f"\nR-squared: {model.rsquared:.4f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"F-statistic: {model.fvalue:.4f}")
print(f"Prob (F-statistic): {model.f_pvalue:.2e}")

print(f"\n💡 해석:")
print(f"- R-squared = {model.rsquared:.2%}: 모델이 구매 금액 변동의 {model.rsquared:.1%}를 설명")
print(f"- 모델의 F-검정 p-value < 0.05: 모델이 통계적으로 유의함")

---

## 2.2 잔차 분석

### 회귀 모델 가정 검증

좋은 회귀 모델은 다음 조건을 만족해야 합니다:
1. **선형성**: 잔차가 무작위로 분포
2. **정규성**: 잔차가 정규분포
3. **등분산성**: 잔차의 분산이 일정

In [ ]:
# 잔차 분석
residuals = model.resid
fitted_values = model.fittedvalues

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('잔차 vs 예측값', '잔차 Q-Q Plot'))

# 잔차 vs 예측값 (등분산성 확인)
fig.add_trace(
    go.Scatter(x=fitted_values, y=residuals, mode='markers', 
               marker=dict(color='royalblue', opacity=0.6)),
    row=1, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)

# Q-Q plot (정규성 확인)
qq_data = probplot(residuals)
fig.add_trace(
    go.Scatter(x=qq_data[0][0], y=qq_data[0][1], mode='markers',
               marker=dict(color='royalblue', opacity=0.6)),
    row=1, col=2
)
x_line = np.linspace(min(qq_data[0][0]), max(qq_data[0][0]), 100)
y_line = qq_data[1][0] * x_line + qq_data[1][1]
fig.add_trace(
    go.Scatter(x=x_line, y=y_line, mode='lines', line=dict(color='red')),
    row=1, col=2
)

fig.update_layout(height=400, width=900, title_text="잔차 분석", showlegend=False)
fig.update_xaxes(title_text="예측값", row=1, col=1)
fig.update_yaxes(title_text="잔차", row=1, col=1)
fig.update_xaxes(title_text="이론적 분위수", row=1, col=2)
fig.update_yaxes(title_text="표본 분위수", row=1, col=2)
fig.show()

# 잔차 정규성 검정
stat, p = stats.shapiro(residuals)
print(f"\n잔차 정규성 검정 (Shapiro-Wilk):")
print(f"  p-value: {p:.4f}")
if p >= 0.05:
    print("  ✅ 잔차가 정규분포를 따름")
else:
    print("  ⚠️ 잔차가 정규분포를 따르지 않음 (큰 표본에서는 일반적)")

---

## 2.3 비즈니스 인사이트 도출

### 분석 결과 종합 리포트

In [ ]:
# 종합 리포트 생성
print("=" * 70)
print("📊 온라인 쇼핑몰 고객 분석 종합 리포트")
print("=" * 70)

print("\n[1. 데이터 개요]")
print(f"  - 분석 대상: {len(customers)}명의 고객")
print(f"  - 평균 구매 금액: {customers['purchase_amount'].mean():,.0f}원")
print(f"  - 구매 금액 범위: {customers['purchase_amount'].min():,}원 ~ {customers['purchase_amount'].max():,}원")

print("\n[2. 주요 발견]")

# 성별 분석 결과
t_stat_gender, p_gender = stats.ttest_ind(male_purchase, female_purchase)
if p_gender < 0.05:
    print(f"  ✅ 성별 차이: 여성이 남성보다 평균 {female_purchase.mean() - male_purchase.mean():,.0f}원 더 구매 (p={p_gender:.4f})")
else:
    print(f"  ❌ 성별 차이: 통계적으로 유의하지 않음 (p={p_gender:.4f})")

# 연령대 분석 결과
f_stat_age, p_age = stats.f_oneway(*groups)
if p_age < 0.05:
    top_age = age_groups.mean().idxmax()
    print(f"  ✅ 연령대 차이: {top_age}가 가장 높은 구매 금액 (p={p_age:.4f})")
else:
    print(f"  ❌ 연령대 차이: 통계적으로 유의하지 않음 (p={p_age:.4f})")

# 상관관계 결과
r_visit, p_visit = stats.pearsonr(customers['visit_count'], customers['purchase_amount'])
print(f"  ✅ 방문 횟수 상관: r={r_visit:.3f} (방문이 많을수록 구매 증가)")

print("\n[3. 핵심 영향 요인 (회귀분석)]")
# 유의한 변수만 추출
sig_vars = coef_df[coef_df['significant']].sort_values('coefficient', ascending=False)
for _, row in sig_vars.iterrows():
    effect = "증가" if row['coefficient'] > 0 else "감소"
    print(f"  - {row['variable_kr']}: {abs(row['coefficient']):,.0f}원 {effect}")

print(f"\n[4. 모델 설명력]")
print(f"  - R-squared: {model.rsquared:.1%} (모델이 구매 금액 변동의 {model.rsquared:.1%} 설명)")

print("\n[5. 비즈니스 제언]")
print("  1. VIP/골드 멤버십 가입 유도 프로모션 강화")
print("  2. 방문 횟수 증가를 위한 재방문 인센티브 프로그램")
print("  3. 체류시간 증가를 위한 콘텐츠 및 UX 개선")
print("  4. 여성 고객 타겟 마케팅 강화")

print("\n" + "=" * 70)

---

## 2.4 범주형 변수 분석 (Chi-Square)

### 질문 5: 멤버십 등급과 구매 여부는 관련이 있는가?

In [ ]:
# Chi-Square 검정: 멤버십 vs 구매 여부
contingency_table = pd.crosstab(customers['membership'], customers['purchased'])
contingency_table.columns = ['미구매', '구매']

print("=" * 60)
print("멤버십 등급 vs 구매 여부 분석")
print("=" * 60)

print("\n교차표:")
print(contingency_table)

# 구매율 계산
purchase_rate = contingency_table['구매'] / contingency_table.sum(axis=1) * 100
print("\n멤버십별 구매율:")
for membership, rate in purchase_rate.items():
    print(f"  {membership}: {rate:.1f}%")

# Chi-square 검정
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nChi-square 검정:")
print(f"  Chi-square 통계량: {chi2:.4f}")
print(f"  p-value: {p_value:.4f}")
print(f"  자유도: {dof}")

print("\n" + "=" * 60)
if p_value < 0.05:
    print("✅ 결론: 멤버십 등급과 구매 여부는 유의한 관련이 있습니다!")
    print(f"   → VIP 멤버십의 구매율이 가장 높음")
else:
    print("❌ 결론: 멤버십 등급과 구매 여부는 독립적입니다.")
print("=" * 60)

In [ ]:
# 멤버십별 구매율 시각화
purchase_rate_df = pd.DataFrame({
    '멤버십': purchase_rate.index,
    '구매율': purchase_rate.values
})

fig = px.bar(
    purchase_rate_df,
    x='멤버십',
    y='구매율',
    color='멤버십',
    title='멤버십 등급별 구매율',
    labels={'구매율': '구매율 (%)'},
    text='구매율',
    category_orders={'멤버십': ['일반', '실버', '골드', 'VIP']}
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(width=600, height=450, showlegend=False)
fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 정규성 검정 ⭐

**문제**: 다음 데이터가 정규분포를 따르는지 Shapiro-Wilk 검정으로 확인하세요.

```python
daily_revenue = [152, 148, 155, 160, 145, 158, 162, 150, 153, 157,
                 149, 156, 151, 154, 159, 147, 161, 150, 155, 152]
```

**기대 결과**: 검정 통계량, p-value, 정규분포 여부 판정

In [ ]:
daily_revenue = [152, 148, 155, 160, 145, 158, 162, 150, 153, 157,
                 149, 156, 151, 154, 159, 147, 161, 150, 155, 152]

# 여기에 코드를 작성하세요


### Q2. 독립 표본 t-test ⭐⭐

**문제**: 온라인 채널과 오프라인 채널의 평균 주문 금액에 유의한 차이가 있는지 검정하세요.

```python
online_orders = [85000, 92000, 78000, 95000, 88000, 91000, 82000, 89000, 94000, 87000]
offline_orders = [72000, 68000, 75000, 70000, 73000, 69000, 74000, 71000, 76000, 67000]
```

**기대 결과**: 각 채널 평균, t-statistic, p-value, 결론

In [ ]:
online_orders = [85000, 92000, 78000, 95000, 88000, 91000, 82000, 89000, 94000, 87000]
offline_orders = [72000, 68000, 75000, 70000, 73000, 69000, 74000, 71000, 76000, 67000]

# 여기에 코드를 작성하세요


### Q3. 대응 표본 t-test ⭐⭐

**문제**: 프로모션 전후 고객 만족도 점수에 유의한 변화가 있는지 검정하세요.

```python
before_promo = [3.5, 3.8, 3.2, 4.0, 3.6, 3.4, 3.9, 3.3, 3.7, 3.5]
after_promo = [4.2, 4.5, 3.9, 4.6, 4.3, 4.1, 4.7, 4.0, 4.4, 4.2]
```

**기대 결과**: 평균 변화량, t-statistic, p-value, 결론

In [ ]:
before_promo = [3.5, 3.8, 3.2, 4.0, 3.6, 3.4, 3.9, 3.3, 3.7, 3.5]
after_promo = [4.2, 4.5, 3.9, 4.6, 4.3, 4.1, 4.7, 4.0, 4.4, 4.2]

# 여기에 코드를 작성하세요


### Q4. ANOVA 분석 ⭐⭐⭐

**문제**: 3개 광고 플랫폼(유튜브, 인스타그램, 페이스북)의 클릭률에 차이가 있는지 ANOVA로 검정하세요.

```python
youtube_ctr = [3.2, 3.5, 3.8, 3.1, 3.6, 3.4]
instagram_ctr = [4.5, 4.8, 4.2, 4.6, 4.9, 4.4]
facebook_ctr = [2.8, 2.5, 3.0, 2.7, 2.9, 2.6]
```

**기대 결과**: 각 플랫폼 평균, F-statistic, p-value, 최고/최저 성과 플랫폼

In [ ]:
youtube_ctr = [3.2, 3.5, 3.8, 3.1, 3.6, 3.4]
instagram_ctr = [4.5, 4.8, 4.2, 4.6, 4.9, 4.4]
facebook_ctr = [2.8, 2.5, 3.0, 2.7, 2.9, 2.6]

# 여기에 코드를 작성하세요


### Q5. Chi-Square 검정 ⭐⭐⭐

**문제**: 디바이스 유형(모바일, PC, 태블릿)과 구매 완료 여부가 독립적인지 검정하세요.

```python
# 교차표: 행(디바이스), 열(구매 완료 여부)
device_purchase = np.array([
    [180, 320],  # 모바일: 구매O, 구매X
    [120, 280],  # PC
    [60, 140]    # 태블릿
])
```

**기대 결과**: 디바이스별 구매율, Chi-square 통계량, p-value, 결론

In [ ]:
device_purchase = np.array([
    [180, 320],  # 모바일: 구매O, 구매X
    [120, 280],  # PC
    [60, 140]    # 태블릿
])

# 여기에 코드를 작성하세요


### Q6. 상관분석 및 시각화 ⭐⭐⭐

**문제**: 광고비와 매출 데이터의 상관관계를 분석하고 산점도로 시각화하세요.

```python
ad_spend = [100, 150, 200, 250, 300, 350, 400, 450, 500, 550]  # 만원
sales = [1200, 1450, 1750, 2100, 2350, 2700, 2950, 3200, 3500, 3800]  # 만원
```

**기대 결과**: 상관계수, p-value, 상관 강도 해석, 산점도(trendline 포함)

In [ ]:
ad_spend = [100, 150, 200, 250, 300, 350, 400, 450, 500, 550]  # 만원
sales = [1200, 1450, 1750, 2100, 2350, 2700, 2950, 3200, 3500, 3800]  # 만원

# 여기에 코드를 작성하세요


### Q7. 단순 회귀분석 ⭐⭐⭐⭐

**문제**: 마케팅 비용으로 매출을 예측하는 단순 회귀 모델을 구축하세요.

```python
marketing_data = pd.DataFrame({
    'marketing_cost': [50, 100, 150, 200, 250, 300, 350, 400],
    'revenue': [500, 850, 1100, 1400, 1650, 1950, 2200, 2500]
})
```

**기대 결과**: 회귀식, R-squared, 마케팅 비용 100만원 증가 시 매출 예측 변화

In [ ]:
marketing_data = pd.DataFrame({
    'marketing_cost': [50, 100, 150, 200, 250, 300, 350, 400],
    'revenue': [500, 850, 1100, 1400, 1650, 1950, 2200, 2500]
})

# 여기에 코드를 작성하세요


### Q8. 다중 회귀분석 ⭐⭐⭐⭐

**문제**: 웹사이트 지표로 전환율을 예측하는 다중 회귀 모델을 구축하세요.

```python
web_data = pd.DataFrame({
    'page_views': [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500],
    'avg_time': [2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0],
    'bounce_rate': [65, 60, 55, 50, 45, 40, 35, 30, 25, 20],
    'conversion_rate': [1.2, 1.8, 2.3, 2.9, 3.4, 4.0, 4.5, 5.1, 5.6, 6.2]
})
```

**기대 결과**: 회귀 결과 요약, R-squared, 가장 영향력 있는 변수

In [ ]:
web_data = pd.DataFrame({
    'page_views': [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500],
    'avg_time': [2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0],
    'bounce_rate': [65, 60, 55, 50, 45, 40, 35, 30, 25, 20],
    'conversion_rate': [1.2, 1.8, 2.3, 2.9, 3.4, 4.0, 4.5, 5.1, 5.6, 6.2]
})

# 여기에 코드를 작성하세요


### Q9. 종합 분석 - 제품 판매 데이터 ⭐⭐⭐⭐⭐

**문제**: 제품 판매 데이터를 종합적으로 분석하세요.

```python
products = pd.DataFrame({
    'product_id': range(1, 51),
    'category': np.random.choice(['전자제품', '의류', '식품'], 50),
    'price': np.random.randint(10000, 100000, 50),
    'ad_cost': np.random.randint(5000, 30000, 50),
    'rating': np.random.uniform(3.0, 5.0, 50).round(1),
    'sales_qty': np.random.randint(50, 500, 50)
})
```

**분석 항목**:
1. 카테고리별 평균 판매량 비교 (ANOVA)
2. 가격과 판매량의 상관관계
3. 판매량 예측 회귀 모델 (가격, 광고비, 평점)
4. 비즈니스 인사이트 도출

In [ ]:
np.random.seed(42)
products = pd.DataFrame({
    'product_id': range(1, 51),
    'category': np.random.choice(['전자제품', '의류', '식품'], 50),
    'price': np.random.randint(10000, 100000, 50),
    'ad_cost': np.random.randint(5000, 30000, 50),
    'rating': np.random.uniform(3.0, 5.0, 50).round(1),
    'sales_qty': np.random.randint(50, 500, 50)
})

# 여기에 코드를 작성하세요


### Q10. 완전 분석 프로젝트 - 고객 세그먼트 분석 ⭐⭐⭐⭐⭐

**문제**: 고객 세그먼트별 행동 패턴을 완전히 분석하세요.

```python
np.random.seed(42)
customer_segments = pd.DataFrame({
    'customer_id': range(1, 101),
    'segment': np.random.choice(['신규', '일반', '충성', 'VIP'], 100, p=[0.3, 0.35, 0.25, 0.1]),
    'age': np.random.randint(20, 60, 100),
    'tenure_months': np.random.randint(1, 60, 100),
    'monthly_spend': np.random.randint(30000, 300000, 100),
    'visit_frequency': np.random.randint(1, 20, 100),
    'satisfaction': np.random.uniform(3.0, 5.0, 100).round(1)
})
```

**분석 항목**:
1. 세그먼트별 기초 통계
2. 세그먼트별 월 지출액 차이 검정 (ANOVA)
3. 방문 빈도와 만족도의 상관관계
4. 월 지출액 예측 회귀 모델
5. 잔차 분석
6. 종합 리포트 및 비즈니스 제언

In [ ]:
np.random.seed(42)
customer_segments = pd.DataFrame({
    'customer_id': range(1, 101),
    'segment': np.random.choice(['신규', '일반', '충성', 'VIP'], 100, p=[0.3, 0.35, 0.25, 0.1]),
    'age': np.random.randint(20, 60, 100),
    'tenure_months': np.random.randint(1, 60, 100),
    'monthly_spend': np.random.randint(30000, 300000, 100),
    'visit_frequency': np.random.randint(1, 20, 100),
    'satisfaction': np.random.uniform(3.0, 5.0, 100).round(1)
})

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 통계 분석 통합 핵심 요약

| 분석 단계 | 목적 | 핵심 도구 |
|----------|------|----------|
| EDA | 데이터 이해 | `describe()`, 히스토그램, 박스플롯 |
| 정규성 검정 | 분석 방법 선택 | `stats.shapiro()`, Q-Q plot |
| 그룹 비교 | 평균 차이 검정 | t-test, ANOVA |
| 독립성 검정 | 범주형 변수 관계 | Chi-square |
| 상관분석 | 변수 간 관계 | Pearson, Spearman |

### Part 2: 종합 분석 프로젝트 핵심 요약

| 분석 기법 | Python 함수 | 결과 해석 |
|----------|------------|----------|
| 다중 회귀 | `smf.ols()` | 회귀계수, R-squared |
| 잔차 분석 | `model.resid` | 모델 가정 검증 |
| 시각화 | `px.scatter()`, `px.bar()` | 인사이트 전달 |

### 💡 실무 팁

1. **분석 순서**: EDA → 정규성 검정 → 적절한 검정 선택 → 결과 해석
2. **정규성 위반 시**: 비모수 검정 사용 (Mann-Whitney, Kruskal-Wallis)
3. **다중 비교**: ANOVA에서 유의하면 사후 검정(Tukey HSD) 수행
4. **상관 ≠ 인과**: 상관관계만으로 원인-결과 주장 금지
5. **모델 검증**: 항상 잔차 분석으로 가정 확인
6. **비즈니스 연결**: 통계 결과를 비즈니스 인사이트로 변환
7. **시각화 중요**: 숫자보다 그래프가 이해관계자 설득에 효과적

### 📈 분석 방법 선택 가이드

```
비교 대상 확인
    ↓
그룹 수 확인
    ├─ 1개 그룹 vs 기준값 → 단일 표본 t-test
    ├─ 2개 그룹
    │     ├─ 독립 → 독립 표본 t-test
    │     └─ 대응 → 대응 표본 t-test
    └─ 3개 이상 그룹 → ANOVA

범주형 변수 관계 → Chi-square

연속형 변수 관계 → 상관분석

예측 모델 필요 → 회귀분석
```